# Point Cloud Layer Example

This notebook demonstrates the DeckGL PointCloudLayer for visualizing 3D point data.

Point cloud layers are ideal for:
- LiDAR data visualization
- 3D scanning results
- Elevation data
- Any 3D point dataset

## Basic Point Cloud with MapLibre

In [ ]:
import random
from anymap_ts import MapLibreMap


# Generate sample 3D point cloud data
def generate_points(center_lng, center_lat, num_points=1000, spread=0.01):
    points = []
    for _ in range(num_points):
        lng = center_lng + random.uniform(-spread, spread)
        lat = center_lat + random.uniform(-spread, spread)
        # Elevation based on distance from center (creates a dome shape)
        dist = ((lng - center_lng) ** 2 + (lat - center_lat) ** 2) ** 0.5
        elevation = max(0, 500 - dist * 50000) + random.uniform(0, 50)
        # Color by elevation (blue to red)
        color_ratio = min(elevation / 500, 1)
        color = [
            int(255 * color_ratio),
            int(100 * (1 - color_ratio)),
            int(255 * (1 - color_ratio)),
            255,
        ]
        points.append({"position": [lng, lat, elevation], "color": color})
    return points


# Generate points around San Francisco
point_cloud = generate_points(-122.4194, 37.7749)

m = MapLibreMap(center=[-122.4194, 37.7749], zoom=14, pitch=60)
m.add_basemap("CartoDB.DarkMatter")
m.add_point_cloud_layer(
    data=point_cloud,
    name="pointcloud-demo",
    get_position="position",
    get_color="color",
    point_size=3,
    opacity=0.9,
)
m

## Point Cloud with DeckGLMap

In [ ]:
from anymap_ts import DeckGLMap

# Generate larger point cloud
large_cloud = generate_points(-122.4194, 37.7749, num_points=5000, spread=0.02)

m2 = DeckGLMap(center=[-122.4194, 37.7749], zoom=13, pitch=50, bearing=30)
m2.add_basemap("CartoDB.DarkMatter")
m2.add_point_cloud_layer(
    data=large_cloud,
    name="large-cloud",
    get_position="position",
    get_color="color",
    point_size=2,
    size_units="pixels",
    opacity=0.8,
)
m2

## Point Cloud with Uniform Color

In [ ]:
# Simple point cloud with uniform color
simple_points = [
    {"position": [lng, lat, alt]}
    for lng in [-122.42, -122.41, -122.40, -122.39]
    for lat in [37.77, 37.78, 37.79, 37.80]
    for alt in [0, 100, 200, 300]
]

m3 = MapLibreMap(center=[-122.405, 37.785], zoom=13, pitch=60)
m3.add_basemap("CartoDB.Positron")
m3.add_point_cloud_layer(
    data=simple_points,
    name="grid-points",
    get_position="position",
    get_color=[0, 200, 255, 200],  # Uniform cyan color
    point_size=8,
)
m3

## Point Cloud with Layer Control

In [ ]:
# Generate two point clouds
cloud1 = generate_points(-122.43, 37.78, num_points=500, spread=0.005)
cloud2 = generate_points(-122.40, 37.77, num_points=500, spread=0.005)

m4 = MapLibreMap(center=[-122.415, 37.775], zoom=13, pitch=45)
m4.add_basemap("CartoDB.DarkMatter")

# Add first point cloud (blue theme)
for p in cloud1:
    p["color"] = [100, 150, 255, 200]

m4.add_point_cloud_layer(
    data=cloud1,
    name="pointcloud-north",
    get_position="position",
    get_color="color",
    point_size=4,
)

# Add second point cloud (orange theme)
for p in cloud2:
    p["color"] = [255, 150, 100, 200]

m4.add_point_cloud_layer(
    data=cloud2,
    name="pointcloud-south",
    get_position="position",
    get_color="color",
    point_size=4,
)

# Add layer control
m4.add_layer_control()
m4

## Combined Arc and Point Cloud Layers

In [ ]:
# Combine arc and point cloud layers
m5 = MapLibreMap(center=[-122.4, 37.78], zoom=11, pitch=50)
m5.add_basemap("CartoDB.DarkMatter")

# Add point clouds at locations
locations = [
    (-122.45, 37.80),  # Golden Gate
    (-122.40, 37.78),  # Downtown
    (-122.35, 37.75),  # East Bay
]

for i, (lng, lat) in enumerate(locations):
    cloud = generate_points(lng, lat, num_points=200, spread=0.003)
    for p in cloud:
        p["color"] = [100 + 50 * i, 200 - 50 * i, 150, 200]
    m5.add_point_cloud_layer(
        data=cloud,
        name=f"pointcloud-loc{i}",
        get_position="position",
        get_color="color",
        point_size=3,
    )

# Add arcs connecting the locations
arcs = [
    {"source": list(locations[0]), "target": list(locations[1])},
    {"source": list(locations[1]), "target": list(locations[2])},
    {"source": list(locations[0]), "target": list(locations[2])},
]

m5.add_arc_layer(
    data=arcs,
    name="arc-connections",
    get_source_color=[0, 255, 200, 255],
    get_target_color=[255, 200, 0, 255],
    get_width=2,
)

m5.add_layer_control()
m5

## Remove Point Cloud Layer

In [ ]:
# Remove a specific point cloud layer
m4.remove_point_cloud_layer("pointcloud-north")

## Export to HTML

In [ ]:
m.to_html("pointcloud_layer_example.html")